In [1]:
import os
os.chdir("/home/jovyan/work")

from src.utils import send_request

## Create Season Files from MSF

### Season 2017/18

In [2]:
result1718 = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games.json")

In [4]:
result1718.keys()

dict_keys(['lastUpdatedOn', 'games', 'references'])

In [6]:
result1718.get("games")[0]

{'schedule': {'attendance': None,
  'awayTeam': {'abbreviation': 'BOS', 'id': 82},
  'broadcasters': [],
  'delayedOrPostponedReason': None,
  'endedTime': None,
  'homeTeam': {'abbreviation': 'CLE', 'id': 86},
  'id': 42070,
  'officials': [],
  'originalStartTime': None,
  'playedStatus': 'COMPLETED',
  'scheduleStatus': 'NORMAL',
  'startTime': '2017-10-18T00:00:00.000Z',
  'venue': {'id': 80, 'name': 'Quicken Loans Arena'},
  'venueAllegiance': 'HOME',
  'weather': None},
 'score': {'awayScoreTotal': 99,
  'currentIntermission': None,
  'currentQuarter': None,
  'currentQuarterSecondsRemaining': None,
  'homeScoreTotal': 102,
  'quarters': [{'awayScore': 19, 'homeScore': 29, 'quarterNumber': 1},
   {'awayScore': 19, 'homeScore': 25, 'quarterNumber': 2},
   {'awayScore': 33, 'homeScore': 18, 'quarterNumber': 3},
   {'awayScore': 28, 'homeScore': 30, 'quarterNumber': 4}]}}